# Solving Systems Of Equations Using Python
Machines have been solving math problems since their creation. Python, being one of the primary languages used for mathematical programming, has a number of tools to help us out. In this notebook we will look at how one can solve a system of equations using python. 

While there are many techniques for solving such a problem, we are interested in one of the easiest and most human friendly approaches. We will examine SymPy which offers a Solvers function which is capable of solving euqations defined using a symbolic language. A symbolic language solver is such that a user can specify an equation as a string like "x = 2 + 5" and the solver will understand and solve the equation and give us the value of x.

In [1]:
# Import the python library that has the objects and functions required for solving symbolic expressions
import sympy

## Example 1: A basic system of two equations

Our system of equations is as follows:

$$ y = x + 2 $$
$$ x + y = 4 $$

The solution is x=1, y=3

In [2]:
# Define our symbols in our equations
x = sympy.Symbol('x')
y = sympy.Symbol('y')

Our SymPy solver requires that the equations being solved are set so they equal zero. So our original equations become:

$$ y - x - 2 = 0 $$
$$ y + x - 4 = 0 $$

In [3]:
# Define our equations such that they equal 0
eq1 = y - x - 2
eq2 = y + x - 4
eqs = [eq1, eq2]

In [4]:
# Solve our equation
answer = sympy.solvers.solve(eqs, x, y)
print("The value of x is {0}".format(answer[x]))
print("The value of y is {0}".format(answer[y]))

The value of x is 1
The value of y is 3


## Example 2: Using a constant
In some cases we may have a system of equations that relies on a constant. For example:

$$ y = x + c $$
$$ x + y = 4 $$

Where $c=6$

In [5]:
x = sympy.Symbol('x')
y = sympy.Symbol('y')
c = 6
eqs = [
    y - x - c,
    y + x - 4
]
answer = sympy.solvers.solve(eqs, x, y)
print("The value of x is {0}".format(answer[x]))
print("The value of y is {0}".format(answer[y]))

The value of x is -1
The value of y is 5


## Example 3: Using simple functions
In some cases we may have a system of equations that relies on functions. For example:

$$ y = x + f(x) $$
$$ x + y = 4 $$

Where f(x) is a function such that $f(x)=2x + 3$

In [6]:
# Define our symbols in our equations
x = sympy.Symbol('x')
y = sympy.Symbol('y')

In [7]:
# Define our function as an extension of the base Function class
#    https://docs.sympy.org/latest/modules/functions/index.html
#

class f(sympy.Function):
 
    # Specify the parameters and logic of the function
    @classmethod
    def eval(self, x):
        return 2 * x + 3

print("The function: {0}".format(f(x)))
print("The derivative: {0}".format(f(x).diff(x)))
print("A solution: {0}".format(f(4)))

The function: 2*x + 3
The derivative: 2
A solution: 11


In [8]:
# Define our equations such that they equal 0
equations =[
    y - x - f(x),
    y + x - 4
]

In [9]:
# Solve our equation
answer = sympy.solvers.solve(equations, x, y, f)
print("The value of x is {0}".format(answer[x]))
print("The value of y is {0}".format(answer[y]))

The value of x is 1/4
The value of y is 15/4


## Example 4: Passing expressions to functions
In some cases we may want to pass expressions to functions:

$$ y = x + f( 2 + x) $$
$$ x + y = 4 $$

Where f(x) is a function such that $f(x)=x + 3$

In [10]:
x = sympy.Symbol('x')
y = sympy.Symbol('y')

class f(sympy.Function):
 
    # Specify the parameters and logic of the function
    @classmethod
    def eval(self, x):
        return 2* x + 3

equations =[
    y - x - f(3 + x),
    y + x - 4
]
answer = sympy.solvers.solve(equations, x, y, f)
print("The value of x is {0}".format(answer[x]))
print("The value of y is {0}".format(answer[y]))

The value of x is -5/4
The value of y is 21/4


## Example 5: Passing expressions to numpy functions
In some cases we may want to pass expressions to functions:

$$ y = x + \beta(2 + x, 1 - x) $$
$$ x + y = 4 $$

Where $ \beta(a,b) = \int_0^1 x^{a-1} (1 - x)^{b-1} dx $

There is an implimentation of the beta function in the scipy library. 

In [11]:
import scipy.special

In [12]:
scipy.special.beta(2, 3)

0.08333333333333333

But we cannot use this implimentation because it does not know how to handle expressions like 2 + x. As such we will not be able to solve the system of equations.

In [13]:
x = sympy.Symbol('x')
y = sympy.Symbol('y')

class B(sympy.Function):
 
    # Specify the parameters and logic of the function
    @classmethod
    def eval(self, a, b):
        try:
            scipy.special.beta(a, b)
        except:
            print("The function broke as expected")

equations =[
    y - x - B(2+x,1-x),
    y + x - 4
]


#answer = sympy.solvers.solve(equations, x, y, f)
#print("The value of x is {0}".format(answer[x]))
#print("The value of y is {0}".format(answer[y]))

The function broke as expected


We could try to define own beta function using sympy primitives.

In [14]:
x = sympy.Symbol('x')
a = sympy.Symbol('a')
b = sympy.Symbol('b')

expression = x**(a - 1) * (1 - x)**(b - 1)
#beta = sympy.integrate(expression, (x, 0, 1))
#beta

But we will see that sympy cannot integrate the expression in this form and the integrate function call seems to hang.

Luckily sympy impliments a beta function. It can be used in expression solving and can render aproximations.

https://docs.sympy.org/latest/modules/functions/special.html#sympy.functions.special.beta_functions.beta

In [15]:
sympy.beta(2,3)

beta(2, 3)

In [16]:
sympy.beta(2,3).evalf()

0.0833333333333333

So we can use the solver as follows

In [17]:
x = sympy.Symbol('x')
y = sympy.Symbol('y')

equations =[
    y - x - sympy.beta(2, 3),
    y - 6
]
answer = sympy.solvers.solve(equations, x, y)
answer

{x: 6 - beta(2, 3), y: 6}

In [18]:
answer[x].evalf()

5.91666666666667

## Example 6: Nested expressions
In some cases we may want to pass expressions to functions:

$$ y = x + z $$
$$ x + y = 4 $$

Where $ z = 3x + 5 $

In [19]:
x = sympy.Symbol('x')
y = sympy.Symbol('y')
z = 3*x + 5

equations =[
    y - x - z,
    x + y - 4
]
answer = sympy.solvers.solve(equations, x, y)
answer

{x: -1/5, y: 21/5}

## Example 7: Solving espressions defined as latex
In some cases we may want to pass expressions to functions:

$$ y = x + z $$
$$ x + y = 4 $$

Where $ z = 3x + 5 $

In [20]:
!pip install antlr4-python3-runtime
import sympy.parsing.latex

You should consider upgrading via the 'c:\program files\python\python 3.8\python.exe -m pip install --upgrade pip' command.


In [21]:
latex1 = r'y - x - z'
eq1 = sympy.parsing.latex.parse_latex(latex1)
latex2 = r'x + y - 4'
eq2 = sympy.parsing.latex.parse_latex(latex2)
latex3 = r'-z + 3*x + 5'
eq3 = sympy.parsing.latex.parse_latex(latex3)

ANTLR runtime and generated code versions disagree: 4.9.1!=4.7.2
ANTLR runtime and generated code versions disagree: 4.9.1!=4.7.2
ANTLR runtime and generated code versions disagree: 4.9.1!=4.7.2
ANTLR runtime and generated code versions disagree: 4.9.1!=4.7.2
ANTLR runtime and generated code versions disagree: 4.9.1!=4.7.2
ANTLR runtime and generated code versions disagree: 4.9.1!=4.7.2


In [22]:
equations =[
    eq1,
    eq2,
    eq3
]
answer = sympy.solvers.solve(equations)
answer

{x: -1/5, y: 21/5, z: 22/5}